# Database Controller Basic Tests

This notebook tests the basic functionality of NAS_DB and VEST_DB controllers.

## Test Objectives:
1. **NAS_DB Tests**:
   - Initialization and connection
   - First call: Data loading from source (with cache creation)
   - Second call: Data loading from cache
   - Force remote fetch (bypass cache)
   - File header retrieval

2. **VEST_DB Tests**:
   - Initialization and connection
   - Shot existence check
   - Data loading
   - Next shot code retrieval

## Configuration:
- Shot numbers and field IDs are configurable in the cells below
- Requires `ifi/config.ini` with proper database configuration



## 1. Setup and Imports


In [1]:
# Setup and imports
import sys
from pathlib import Path
import os

# Configure Numba threading layer for parallel execution
os.environ['NUMBA_THREADING_LAYER'] = 'tbb'

# Add project root to path
current_dir = Path.cwd()
ifi_root = current_dir.parent if current_dir.name == "db_controller" else current_dir
sys.path.insert(0, str(ifi_root))

from ifi.utils.cache_setup import setup_project_cache
cache_config = setup_project_cache()
print(f"Cache configured: {cache_config['cache_dir']}")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import Numba config after setting environment variable
try:
    import numba
    try:
        numba.config.THREADING_LAYER = 'tbb'
        print(f"Numba threading layer: {numba.config.THREADING_LAYER}")
    except Exception as e:
        print(f"Warning: Could not set Numba threading layer: {e}")
        print("Falling back to default threading layer")
except ImportError:
    print("Warning: Numba not available")

# Import IFI modules
from ifi.utils.common import LogManager
from ifi.db_controller.nas_db import NAS_DB
from ifi.db_controller.vest_db import VEST_DB

# Initialize logging
LogManager(level="INFO")
logger = LogManager().get_logger(__name__)

print("✓ All imports successful")


Using cache directory: C:\Users\dhkdw\Documents\mygit\ifi\cache\numba_cache
  Using fallback threading layer: omp
Numba threading layer: omp
Project cache configured successfully.
Cache configured: C:\Users\dhkdw\Documents\mygit\ifi\cache\numba_cache


INFO     | 
[LOGS -START] Logging started

INFO     | 
[LOGS -START] All logs for this execution will be saved to: logs\251106_144909_interactive.log


Numba threading layer: tbb
✓ All imports successful


## 2. Test Configuration


In [2]:
# Test Configuration
# PLEASE ADJUST THESE VALUES BASED ON YOUR 'ifi/config.ini' AND AVAILABLE DATA
SHOT_TO_TEST = 45821  # Test for CSV files, 'MDO3000orig', 'MDO3000fetch', 'MSO58'
# SHOT_TO_TEST = 41715 # Test for CSV files, 'MDO3000orig', 'MDO3000fetch'
# SHOT_TO_TEST = 36853 # Test for CSV files, 'MSO58'
# SHOT_TO_TEST = 38396 # Test for 'MDO3000pc'
# SHOT_TO_TEST = 'AGC w attn I' # Test for 'ETC'

CONFIG_PATH = 'ifi/config.ini'
CACHE_FOLDER = Path('./cache')  # Should match [LOCAL_CACHE] dumping_folder in config

VEST_SHOT_TO_TEST = 40656
VEST_FIELD_TO_TEST = 109

print(f"Test Configuration:")
print(f"  - Config path: {CONFIG_PATH}")
print(f"  - Cache folder: {CACHE_FOLDER}")
print(f"  - NAS shot number: {SHOT_TO_TEST}")
print(f"  - VEST shot number: {VEST_SHOT_TO_TEST}")
print(f"  - VEST field number: {VEST_FIELD_TO_TEST}")


Test Configuration:
  - Config path: ifi/config.ini
  - Cache folder: cache
  - NAS shot number: 45821
  - VEST shot number: 40656
  - VEST field number: 109


## 3. NAS_DB Tests

### Test 1: First Call - Fetch and Cache


In [3]:
# Test 1: First call - Data is not cached. It will be fetched from the remote source
# and then saved to the local cache (e.g., './cache/45821.h5').
logger.info("=" * 60)
logger.info("NAS_DB Basic Functionality Test")
logger.info("=" * 60)

if not Path(CONFIG_PATH).exists():
    logger.error(f"Configuration file not found at '{CONFIG_PATH}'")
    logger.error("Please create it from the template")
else:
    logger.info("\n--- Test 1: First call to get_shot_data (should fetch and cache) ---")
    
    try:
        with NAS_DB(config_path=CONFIG_PATH) as nas:
            data_dict = nas.get_shot_data(SHOT_TO_TEST)
            
            if data_dict and isinstance(data_dict, dict) and len(data_dict) > 0:
                logger.info("   -> Data loaded successfully on first call.")
                logger.info(f"   -> Number of dataframes: {len(data_dict)}")
                for key, df in data_dict.items():
                    logger.info(f"   -> DataFrame '{key}' shape: {df.shape}")
                    logger.info(f"      - Columns: {df.columns.tolist()}")
                
                # Display first file head
                first_df = list(data_dict.values())[0]
                print("\n--- Data Head (first file) ---")
                print(first_df.head())
                print("------------------------------")
            else:
                logger.error("   -> FAILED: Could not retrieve any data on the first call.")
                logger.warning("   This may be expected if shot number is not available or NAS is not accessible")
    except Exception as e:
        logger.error(f"Failed to connect to NAS or load data: {e}", exc_info=True)


KeyboardInterrupt: 

### Test 2: Second Call - Load from Cache


In [ ]:
# Test 2: Second call - The local cache file now exists. This call should be much faster
# as it reads directly from the local HDF5 file.
logger.info("\n--- Test 2: Second call to get_shot_data (should load from cache) ---")

try:
    with NAS_DB(config_path=CONFIG_PATH) as nas:
        cached_data_dict = nas.get_shot_data(SHOT_TO_TEST)
        
        if cached_data_dict and isinstance(cached_data_dict, dict):
            logger.info("   -> Data loaded successfully from cache.")
            logger.info(f"   -> Number of dataframes: {len(cached_data_dict)}")
            for key, df in cached_data_dict.items():
                logger.info(f"   -> DataFrame '{key}' shape: {df.shape}")
            
            if len(cached_data_dict) == len(data_dict):
                logger.info("   -> SUCCESS: Number of cached files matches original files.")
            else:
                logger.error(f"   -> FAILED: Number mismatch: original={len(data_dict)}, cached={len(cached_data_dict)}")
        else:
            logger.error("   -> FAILED: Could not retrieve data from cache or result is not a dictionary.")
except Exception as e:
    logger.error(f"Failed to load from cache: {e}", exc_info=True)


### Test 3: Verify Cache File Creation


In [ ]:
# Test 3: Verify cache file creation
cache_dir = CACHE_FOLDER / str(SHOT_TO_TEST)
expected_cache_file = cache_dir / f"{SHOT_TO_TEST}.h5"

logger.info("\n--- Test 3: Verifying cache file creation ---")
if expected_cache_file.exists():
    logger.info(f"   -> SUCCESS: Cache file created at '{expected_cache_file}'")
    cache_size = expected_cache_file.stat().st_size
    logger.info(f"      - Cache file size: {cache_size:,} bytes")
else:
    logger.warning(f"   -> WARNING: Cache file was not created at '{expected_cache_file}'.")
    logger.warning("   This may be expected if caching is disabled or connection failed")


### Test 4: Force Remote Fetch


In [ ]:
# Test 4: Force remote fetch - Use 'force_remote=True' to bypass the local cache and
# re-download the data from the source. This is useful if the data has changed.
logger.info("\n--- Test 4: Third call with force_remote=True (should bypass cache) ---")

try:
    with NAS_DB(config_path=CONFIG_PATH) as nas:
        forced_data_dict = nas.get_shot_data(SHOT_TO_TEST, force_remote=True)
        
        if forced_data_dict and isinstance(forced_data_dict, dict):
            logger.info("   -> Data loaded successfully by forcing remote fetch.")
            logger.info(f"   -> Number of dataframes: {len(forced_data_dict)}")
            for key, df in forced_data_dict.items():
                logger.info(f"   -> DataFrame '{key}' shape: {df.shape}")
        else:
            logger.warning("   -> WARNING: Could not load data with force_remote=True.")
            logger.warning("   This may be expected if remote connection is not available")
except Exception as e:
    logger.error(f"Failed to force remote fetch: {e}", exc_info=True)


### Test 5: File Header Retrieval


In [ ]:
# Test 5: File header retrieval
logger.info(f"\n--- Test 5: Fetching top lines for shot #{SHOT_TO_TEST}... ---")

try:
    with NAS_DB(config_path=CONFIG_PATH) as nas:
        file_head = nas.get_data_top(SHOT_TO_TEST, lines=50)
        
        if file_head:
            logger.info("   -> SUCCESS: Retrieved file head.")
            print("\n--- File Head ---")
            print(file_head)
            print("-----------------")
        else:
            logger.warning("   -> WARNING: Could not retrieve file head.")
            logger.warning("   This may be expected if method is not available or file not found")
except Exception as e:
    logger.error(f"Failed to retrieve file head: {e}", exc_info=True)



--- File Head ---
Model,MDO3014
Firmware Version,empty
,
Waveform Type,empty
Point Format,Y
Horizontal Units,s
Horizontal Scale,empty
Horizontal Delay,empty
Sample Interval,4e-09
Record Length,10000000
Gating,empty
Probe Attenuation,empty,empty,empty
Vertical Units,V,V,V
Vertical Offset,-2,0.0026,-0.0124
Vertical Scale,empty,empty,empty
Vertical Position,empty,empty,empty
,,,
,,,
,,,
Label,,,
TIME,CH1,CH2,CH3
-0.01,-0.48,0.0186,0.014
-0.009999996,-0.72,0.0206,0.0156
-0.009999992,-0.72,0.021,0.0184
-0.009999988,-0.56,0.0206,0.0196
-0.009999984,-0.16,0.019,0.0208
-0.00999998,0.16,0.0174,0.02
-0.009999976,0.4,0.0158,0.0196
-0.009999972,0.72,0.0142,0.0172
-0.009999968,0.72,0.0126,0.0164
-0.009999964,0.56,0.0134,0.0144
-0.00999996,0.4,0.015,0.0132
-0.009999956,0,0.0158,0.0132
-0.009999952,-0.32,0.0174,0.0132
-0.009999948,-0.64,0.0198,0.0152
-0.009999944,-0.72,0.0198,0.0172
-0.00999994,-0.72,0.021,0.0192
-0.009999936,-0.4,0.0198,0.0204
-0.009999932,0,0.0186,0.0204
-0.009999928,0.4,0.017,0.0

## 4. VEST_DB Tests

### Test 1: Check Shot Existence


In [ ]:
# Test 1: Check if shot exists
logger.info("=" * 60)
logger.info("VEST_DB Basic Functionality Test")
logger.info("=" * 60)

if not Path(CONFIG_PATH).exists():
    logger.error(f"Configuration file not found at '{CONFIG_PATH}'")
    logger.error("Please create it from the template")
else:
    logger.info(f"\n--- Test 1: Checking for existence of VEST shot #{VEST_SHOT_TO_TEST}, Field #{VEST_FIELD_TO_TEST}... ---")
    
    try:
        with VEST_DB(config_path=CONFIG_PATH) as db:
            existence = db.exist_shot(VEST_SHOT_TO_TEST, VEST_FIELD_TO_TEST)
            
            if existence > 0:
                logger.info(f"   -> SUCCESS: Shot found in table (shotDataWaveform_{existence}).")
            else:
                logger.warning(f"   -> WARNING: Shot #{VEST_SHOT_TO_TEST} not found in VEST DB.")
                logger.warning("   This may be expected if test shot number doesn't exist")
    except Exception as e:
        logger.error(f"Failed to connect to VEST database: {e}", exc_info=True)


### Test 2: Load Shot Data


In [4]:
# Test 2: Load shot data
logger.info(f"\n--- Test 2: Attempting to load data for shot #{VEST_SHOT_TO_TEST}, Field #{VEST_FIELD_TO_TEST}... ---")

try:
    with VEST_DB(config_path=CONFIG_PATH) as db:
        result_dict = db.load_shot(VEST_SHOT_TO_TEST, [VEST_FIELD_TO_TEST])
        
        if result_dict and isinstance(result_dict, dict) and len(result_dict) > 0:
            logger.info("   -> SUCCESS: VEST data loaded successfully.")
            for rate_key, df in result_dict.items():
                logger.info(f"   -> Rate group '{rate_key}':")
                logger.info(f"      - DataFrame shape: {df.shape}")
                logger.info(f"      - Columns: {df.columns.tolist()}")
                
                print(f"\n--- Head of DataFrame ({rate_key}) ---")
                print(df.head())
                print("-------------------------")
        else:
            logger.error("   -> FAILED: Could not load VEST data or result is empty.")
except Exception as e:
    logger.error(f"Failed to load VEST data: {e}", exc_info=True)



--- Head of DataFrame (25k) ---
         Ip_raw ([V])
0.00000      0.015717
0.00004      0.018311
0.00008      0.020142
0.00012      0.018921
0.00016      0.019836
-------------------------


### Test 3: Get Next Shot Code


In [5]:
# Test 3: Get next shot code
logger.info("\n--- Test 3: Getting next shot code... ---")

try:
    with VEST_DB(config_path=CONFIG_PATH) as db:
        next_shot = db.get_next_shot_code()
        if next_shot:
            logger.info(f"   -> SUCCESS: Next shot code: {next_shot}")
        else:
            logger.warning("   -> WARNING: Could not get next shot code.")
except Exception as e:
    logger.error(f"Failed to get next shot code: {e}", exc_info=True)


## 5. Test Summary


In [6]:
# Test Summary
logger.info("\n" + "=" * 60)
logger.info("Test Summary")
logger.info("=" * 60)
logger.info("\nAll tests completed. Check the logs above for detailed results.")
logger.info("\nNote: Some tests may fail if database connections are not available.")
logger.info("This is expected in test environments.")
